In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [ ]:
!cd yolov5 && python train.py --img 320 --batch 16 --epochs 500 --data dataset.yml --weights yolov5s.pt --workers 2 

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp9/weights/last.pt', force_reload=True)

In [ ]:
import os
import time
import uuid

In [ ]:
image_links = {
    'https://previews.123rf.com/images/ammentorp/ammentorp1611/ammentorp161100520/66380796-young-friends-having-fun-on-shopping-carts-multiracial-young-people-racing-on-shopping-cart.jpg',
    'https://images.ctfassets.net/9l3tjzgyn9gr/aR8LRVbCLG8t3AvhK2LQo/3fca3c7325d478c9183a510726afa921/SmartCart_1224.jpg?fm=jpg&fl=progressive&q=50&w=1200',
    'https://www.cincinnati.com/gcdn/-mm-/b11ddaaebd6c5629dbc3a3c97a2349b520951aad/c=0-50-534-351/local/-/media/Cincinnati/Cincinnati/2014/06/25/1403697633000-downtowngrocery1.jpg?width=1200&disable=upscale&format=pjpg&auto=webp',
    'https://townsquare.media/site/84/files/2020/11/GettyImages-129472166.jpg',
    'https://www.drivingdynamics.com/hubfs/iStock-623710682.jpg',
    'https://philressler.com/wp-content/uploads/2022/01/shopping-cart.jpeg'
    
}

# Iterating...
for i in image_links:
    print(i)
    
    results = model(i)
    class_ids = results.pred[0].detach().cpu().numpy()[:, -1].astype(int)
    print(class_ids)
    %matplotlib inline 
    plt.imshow(np.squeeze(results.render()))
    plt.show()
    print("\n")

## real time

In [ ]:
cap = cv2.VideoCapture('mall_two.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

## real time with counter

In [ ]:
# Open the video capture
cap = cv2.VideoCapture('mall_two.mp4')
counter = 0
while cap.isOpened():
    
    if counter < 0:
        break
        
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    # Get the class labels and confidences
    class_ids = results.pred[0].detach().cpu().numpy()[:, -1].astype(int)
    confidences = results.pred[0].detach().cpu().numpy()[:, 4]
    
    # Check if "no owner" class is detected
    if 15 in class_ids:
        counter = counter + 1
    else:
        counter = counter - 1
    
    cv2.putText(canvas, f'Shopping Cart Count: {counter}', (30, 40), cv2.FONT_HERSHEY_SIMPLEX,2, (255, 255, 255), 2)
    # Combine the frame and canvas
    combined = np.hstack((frame, canvas))

    # Display the combined frame
    cv2.imshow(combined)
    # Display the frame
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()
